## ДЗ_4

1. скачать набор данных маркетинговых кампаний отсюда https://www.kaggle.com/davinwijaya/customer-retention
2. поле conversion - это целевая переменная, а offer - коммуникация. Переименовать поля (conversion -> target, offer -> treatment) и привести поле treatment к бинарному виду (1 или 0, т.е было какое-то предложение или нет) - значение No Offer означает отсутствие коммуникации, а все остальные - наличие.
3. сделать разбиение набора данных не тренировочную и тестовую выборки
4. провести uplift-моделирование 3 способами:
    1. одна модель с признаком коммуникации (S-learner)
    2. модель с трансформацией таргета
    3. вариант с двумя независимыми моделями
5. в конце вывести единую таблицу сравнения метрик uplift@10%, uplift@20% 3 моделей
6. *для модели S-learner построить зависимость таргета (конверсии - поле conversion) от значения uplift:
    1. сделать прогноз и получить uplift для тестовой выборки
    2. отсортировать тестовую выборку по uplift по убыванию
    3. разбить на децили (pandas qcut вам в помощь)
    4. для каждого дециля посчитать среднюю conversion

In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('HW_data.csv')
df.head()

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,offer,conversion
0,10,142.44,1,0,Surburban,0,Phone,Buy One Get One,0
1,6,329.08,1,1,Rural,1,Web,No Offer,0
2,7,180.65,0,1,Surburban,1,Web,Buy One Get One,0
3,9,675.83,1,0,Rural,1,Web,Discount,0
4,2,45.34,1,0,Urban,0,Web,Buy One Get One,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64000 entries, 0 to 63999
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   recency        64000 non-null  int64  
 1   history        64000 non-null  float64
 2   used_discount  64000 non-null  int64  
 3   used_bogo      64000 non-null  int64  
 4   zip_code       64000 non-null  object 
 5   is_referral    64000 non-null  int64  
 6   channel        64000 non-null  object 
 7   offer          64000 non-null  object 
 8   conversion     64000 non-null  int64  
dtypes: float64(1), int64(5), object(3)
memory usage: 4.4+ MB


преобразуем колонки 

In [5]:
df.rename(columns={'conversion': 'target', 'offer': 'treatment'}, inplace=True)

In [6]:
df['treatment'] = (df['treatment'] != 'No Offer').astype(int)

Разделим на Train и Test

In [7]:
df_train, df_valid = train_test_split(df, test_size=0.3, random_state=123)

In [8]:
features = ['recency', 'history', 'used_discount', 'used_bogo', 'zip_code', 'is_referral', 'channel']
cat_features = ['zip_code', 'channel']

In [9]:
X_train = df_train[features]
y_train = df_train['target']
treat_train = df_train['treatment']

X_valid = df_valid[features]
y_valid = df_valid['target']
treat_valid = df_valid['treatment']

In [13]:
pip install scikit-uplift catboost scikit-learn seaborn matplotlib pandas numpy

Note: you may need to restart the kernel to use updated packages.


In [14]:
from sklift.metrics import uplift_at_k
from sklift.viz import plot_uplift_preds
from sklift.models import SoloModel

from catboost import CatBoostClassifier

In [15]:
models_results = {
    'approach': [],
    'uplift@20%': [],
    'uplift@10%': []
}

Одиночная модель

In [16]:
sm = SoloModel(
    CatBoostClassifier(iterations=20, random_state=42, silent=True, cat_features=cat_features)
)

sm = sm.fit(X_train, y_train, treat_train)

uplift_sm = sm.predict(X_valid)

sm_score_20 = uplift_at_k(y_true=y_valid, uplift=uplift_sm, treatment=treat_valid, strategy='by_group', k=0.2)
sm_score_10 = uplift_at_k(y_true=y_valid, uplift=uplift_sm, treatment=treat_valid, strategy='by_group', k=0.1)

models_results['approach'].append('SoloModel')
models_results['uplift@20%'].append(sm_score_20)
models_results['uplift@10%'].append(sm_score_10)

С двумя моделями

In [18]:
from sklift.models import TwoModels


tm = TwoModels(
    CatBoostClassifier(iterations=20, random_state=42, silent=True, cat_features=cat_features),
    CatBoostClassifier(iterations=20, random_state=42, silent=True, cat_features=cat_features),
    method='vanilla'  # независимые модели
)
tm = tm.fit(
    X_train, y_train, treat_train
)

uplift_tm = tm.predict(X_valid)

tm_score_20 = uplift_at_k(y_true=y_valid, uplift=uplift_tm, treatment=treat_valid, strategy='by_group', k=0.2)
tm_score_10 = uplift_at_k(y_true=y_valid, uplift=uplift_tm, treatment=treat_valid, strategy='by_group', k=0.1)

models_results['approach'].append('TwoModels')
models_results['uplift@20%'].append(tm_score_20)
models_results['uplift@10%'].append(tm_score_10)

С Трансформацией класса

In [19]:
from sklift.models import ClassTransformation


ct = ClassTransformation(
    CatBoostClassifier(iterations=20, random_state=42, silent=True, cat_features=cat_features)
)
ct = ct.fit(X_train, y_train, treat_train)

uplift_ct = ct.predict(X_valid)

ct_score_20 = uplift_at_k(y_true=y_valid, uplift=uplift_ct, treatment=treat_valid, strategy='by_group', k=0.2)
ct_score_10 = uplift_at_k(y_true=y_valid, uplift=uplift_ct, treatment=treat_valid, strategy='by_group', k=0.1)

models_results['approach'].append('ClassTransformation')
models_results['uplift@20%'].append(ct_score_20)
models_results['uplift@10%'].append(ct_score_10)

Вывод

In [25]:
pd.DataFrame(data=models_results)

,approach,uplift@20%,uplift@10%
0,SoloModel,0.078994,0.089254
1,TwoModels,0.068439,0.078306
2,TwoModels,0.068439,0.078306
3,ClassTransformation,0.095800,0.117411


Трансформация класса показала лучший результат